In [1]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema, PFNanoAODSchema
import json
import fastjet
import numpy as np
import awkward as ak
from coffea import processor
import hist
import coffea.nanoevents.methods.vector as vector
import warnings
import matplotlib.pyplot as plt
import math

/tmp/ipykernel_2551031/1518451047.py:8: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  import coffea.nanoevents.methods.vector as vector


In [2]:
warnings.filterwarnings("ignore", module="coffea.*")
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")
warnings.filterwarnings("ignore", "Please ensure")
warnings.filterwarnings("ignore", "invalid value")

In [3]:
with open('../../filelists/hgg_files.txt', 'r') as f:
    hgg_files = [line.strip() for line in f]

In [4]:
hgg = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/signal/hgg/' + hgg_files[0]: "/Events"},
    delayed=False,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "Hgg"},
).events()

In [92]:
cut = ((hgg.FatJet.pt>300) & 
       (abs(hgg.FatJet.phi)<1) & 
       (abs(hgg.FatJet.eta)<2.5))
fatjet = hgg.FatJet[cut]

In [93]:
pf = ak.flatten(fatjet.constituents.pf, axis=1)

In [94]:
jetdef = fastjet.JetDefinition(fastjet.cambridge_algorithm, 0.2)

In [95]:
cluster = fastjet.ClusterSequence(pf, jetdef)

In [96]:
subjets = cluster.inclusive_jets()

In [97]:
vec = ak.zip(
    {
        "x": subjets.px,
        "y": subjets.py,
        "z": subjets.pz,
        "t": subjets.E,
    },
    with_name="LorentzVector",
    behavior=vector.behavior,
)

In [98]:
vec = ak.pad_none(vec, 3)

In [99]:
vec["idx"] = ak.local_index(vec)

In [100]:
i, j = ak.unzip(ak.combinations(vec, 2))

In [101]:
best = ak.argmax(abs((i + j).pt), axis=1, keepdims=True)

In [102]:
order_check = ak.concatenate([i[best].pt, j[best].pt], axis=1)

In [103]:
leading = ak.argmax(order_check, axis=1, keepdims=True)

In [104]:
subleading = ak.argmin(order_check, axis=1, keepdims=True)

In [105]:
leading_particles = ak.concatenate([i[best], j[best]], axis=1)

In [106]:
cut = ((vec.idx != ak.firsts(leading_particles.idx)) & 
       (vec.idx != ak.firsts(ak.sort(leading_particles.idx, ascending=False)))
      )

In [107]:
everything_else = vec[cut]


In [126]:
total_soft_pt = ak.sum(everything_else.pt, axis=1)

In [127]:
everything_else['momentum_fraction'] = (everything_else.pt)/total_soft_pt

In [110]:
everything_else['weighted_eta'] = everything_else.eta * everything_else.momentum_fraction

In [111]:
everything_else['weighted_phi'] = everything_else.phi * everything_else.momentum_fraction

In [112]:
weighted_average_eta = ak.sum(everything_else.weighted_eta, axis=1)/ak.num(everything_else, axis=1)

In [113]:
average_eta = ak.sum(everything_else.eta, axis=1)/ak.num(everything_else, axis=1)

In [114]:
weighted_average_phi = ak.sum(everything_else.weighted_phi, axis=1)/ak.num(everything_else, axis=1)

In [115]:
average_phi = ak.sum(everything_else.phi, axis=1)/ak.num(everything_else, axis=1)

In [116]:
leg1 = ak.firsts(leading_particles[leading])
leg2 = ak.firsts(leading_particles[subleading])

In [117]:
a13 = ((((leg1.eta * (leg1.pt/total_pt)) - weighted_average_eta)**2) + (((leg1.phi * (leg1.pt/total_pt)) - weighted_average_phi)**2))
a23 = ((((leg2.eta * (leg2.pt/total_pt)) - weighted_average_eta)**2) + (((leg2.phi * (leg2.pt/total_pt)) - weighted_average_phi)**2))
a12 = ((((leg1.eta * (leg1.pt/total_pt)) - (leg2.eta * (leg2.pt/total_pt)))**2) + (((leg1.phi * (leg1.pt/total_pt)) - (leg2.phi * (leg2.pt/total_pt)))**2))
color_ring = (a13 + a23) / (a12)

In [118]:
color_ring

<Array [1.04, 2.84, 3.38, 1.03, ..., 0.866, 13.6, 11.7] type='489 * ?float64'>

In [119]:
everything_else[6].pt

<Array [4.32, 0.673, 10.9, 1.58, ..., 1.03, 1.3, 2.03] type='10 * ?float64'>

In [120]:
leg1[6].pt

238.91807443433095

In [121]:
leg2[6].pt

90.48466409057333

In [128]:
everything_else['momentum_fraction']

<Array [[0.164, 0.0191, ..., 0.175], ...] type='489 * option[var * ?float64]'>